In [2]:
%load_ext autoreload
%autoreload 2
import musicxml_parser
import sys
from pathlib import Path
import torch
sys.path.append('../virtuosoNet/')

In [3]:
score_file = 'Cadenza_3dChopin_v0.musicxml'
xml_obj = musicxml_parser.MusicXMLDocument(score_file)
from src.virtuoso.inference import get_input_from_xml

In [4]:
from src.virtuoso.parser import get_parser, get_name
from src.virtuoso import utils
from src.virtuoso import model as modelzoo, encoder_score as encs
from src.virtuoso import encoder_perf as encp, decoder as dec, residual_selector as res
from src.virtuoso.utils import load_weight
from src.virtuoso.inference import get_input_from_xml
from src.virtuoso.dataset import ScorePerformDataset, MultiplePerformSet
from src.virtuoso.data_process import make_slicing_indexes_by_measure, make_slice_with_same_measure_number, key_augmentation

In [5]:
import shlex
parser = get_parser()
args = parser.parse_args(shlex.split('--session_mode=inference --checkpoint ../virtuosoNet/han_measnote_delta/checkpoint_best.pt'))

In [6]:
hier_type = ['is_hier', 'in_hier', 'hier_beat', 'hier_meas', 'meas_note']
curr_type = [x for x in hier_type if getattr(args, x)]

In [7]:
from datetime import datetime
name = get_name(parser, args)  + "_" + datetime.now().strftime('%y%m%d-%H%M%S')
device='cuda'

In [8]:
net_param = torch.load(args.checkpoint, map_location='cpu')['network_params']
args.yml_path = list(Path(args.checkpoint).parent.rglob('*.yml'))[0]
config = utils.read_model_setting(args.yml_path)
args.graph_keys = net_param.graph_keys
criterion = utils.make_criterion_func(config.train_params.loss_type, device)

In [9]:
model = modelzoo.VirtuosoNet()
model.score_encoder = getattr(encs, net_param.score_encoder_name)(net_param)
model.performance_encoder = getattr(encp, net_param.performance_encoder_name)(net_param)
model.residual_info_selector = getattr(res, net_param.residual_info_selector_name)()
model.performance_decoder = getattr(dec, net_param.performance_decoder_name)(net_param)
model.network_params = net_param
model = model.to(device)

In [10]:
model = load_weight(model, args.checkpoint)

=> loaded checkpoint '../virtuosoNet/han_measnote_delta/checkpoint_best.pt' (epoch 7)


In [11]:
ckp = torch.load(args.checkpoint, map_location='cpu')
stats = ckp['stats']

In [12]:
score, input_x, edges, note_locations = get_input_from_xml(score_file, 'Chopin', None, stats['input_keys'], model.stats['graph_keys'], model.stats['stats'], device)

In [13]:
xml_obj = musicxml_parser.MusicXMLDocument(score_file)

In [33]:
import random
import copy
def modifier(xml_obj, p_add=0.15, p_shift=0.1, p_del = 0.1, modifier=1):
    xml_new = copy.deepcopy(xml_obj)
    
    weights = [el*modifier for el in [p_add, p_shift, p_del]]
    weights.append(1 - sum(weights))
    n_add = 0
    for part in xml_new.parts:
        for n, measure in enumerate(part.measures):
            add_notes = []
            for note in measure.notes:
                if not note.is_rest:
                    action = random.choices(['add', 'shift', 'del', 'pass'], weights=weights)[0]
                    if action == 'add':
                        n_add += 1
                        add_notes.extend(note_add(note))
                    elif action == 'shift':
                        pitch_min = max(21, note.pitch[1] - 24)
                        pitch_max = min(108, note.pitch[1] + 24)
                        pitches = list(range(pitch_min, note.pitch[1])) + list(range(note.pitch[1]+1, pitch_max))
                        selected_pitch = random.sample(pitches, 1)[0]
                        note.pitch = (note.pitch[0], selected_pitch)
                    elif action == 'del':
                        note.is_rest = True
                    else:
                        pass
            measure.notes.extend(add_notes)
    print(n_add)
    return xml_new
                    

def note_add(note):
    add_notes = []
    n_notes = random.choices(range(1,6), weights=reversed(range(5)))[0]
    pitch_min = max(21, note.pitch[1] - 24)
    pitch_max = min(108, note.pitch[1] + 24)
    pitches = list(range(pitch_min, note.pitch[1])) + list(range(note.pitch[1]+1, pitch_max))
    selected_pitch = random.sample(pitches, n_notes)
    for pitch in selected_pitch:
        note_new = copy.deepcopy(note)
        note_new.pitch = (note.pitch[0], pitch)
        add_notes.append(note_new)
    return add_notes
    

In [32]:
import shlex
parser = get_parser()
args = parser.parse_args(shlex.split('--session_mode=inference --checkpoint ../virtuosoNet/han_measnote_delta/checkpoint_best.pt'))

In [16]:
hier_type = ['is_hier', 'in_hier', 'hier_beat', 'hier_meas', 'meas_note']
curr_type = [x for x in hier_type if getattr(args, x)]

In [17]:
from datetime import datetime
name = get_name(parser, args)  + "_" + datetime.now().strftime('%y%m%d-%H%M%S')
device='cuda'

In [18]:
net_param = torch.load(args.checkpoint, map_location='cpu')['network_params']
args.yml_path = list(Path(args.checkpoint).parent.rglob('*.yml'))[0]
config = utils.read_model_setting(args.yml_path)
args.graph_keys = net_param.graph_keys
criterion = utils.make_criterion_func(config.train_params.loss_type, device)

In [19]:
model = modelzoo.VirtuosoNet()
model.score_encoder = getattr(encs, net_param.score_encoder_name)(net_param)
model.performance_encoder = getattr(encp, net_param.performance_encoder_name)(net_param)
model.residual_info_selector = getattr(res, net_param.residual_info_selector_name)()
model.performance_decoder = getattr(dec, net_param.performance_decoder_name)(net_param)
model.network_params = net_param
model = model.to(device)

In [20]:
model = load_weight(model, args.checkpoint)

=> loaded checkpoint '../virtuosoNet/han_measnote_delta/checkpoint_best.pt' (epoch 7)


In [21]:
import pickle
import matplotlib.pyplot as plt
with open('/data/project/pid/notebooks/emotion/emotion_z.dat', 'rb') as f:
    train_z = pickle.load(f)

In [22]:
candidate=[('Clementi.sonatine_op36_no1_mov3.mm_1-end.', 's005', 'E3'), 
           # ('Clementi.sonatine_op36_no1_mov3.mm_1-end.', 's019', 'E3'), 
           ('Bach.prelude-and-fugue_bwv870_no1_prelude.mm_1-19.s007', 's007', 'E2'),
           # ('Liszt.consolation_s172_no3_.mm_1-45', 's007', 'E2'),
           ('Clementi.sonatine_op36_no1_mov3.mm_1-34.', 's001', 'E5'), 
           # ('Mendelssohn.variations', 's019', 'E1'), 
           ('Chopin.valse-brillante_op34_no2_.mm_1-36', 's008', 'E5'),
          ]

n_candi = len(candidate)

In [23]:
labels = [[] for _ in range(n_candi)]
selected_z = [[] for _ in range(n_candi)]
for _, (k, v) in enumerate(train_z.items()):
    pieces, mu, var = zip(*v)
    for n, piece in enumerate(pieces):
        if any([(el[0] in piece and el[1] in piece and str(k) == el[2]) for el in candidate]):
            for idx, info in enumerate(candidate):
                if info[0] in piece and info[1] in piece and str(k) == info[2]:
                    labels[idx] = '.'.join(info)
                    selected_z[idx] = mu[n]


In [34]:
import numpy as np
from src.virtuoso.inference import scale_model_prediction_to_original, model_prediction_to_feature
from src.virtuoso.model_utils import reparameterize
from src.virtuoso.inference import save_model_output_as_midi
xml_obj = musicxml_parser.MusicXMLDocument(score_file)
for k in range(10):
    xml_modified = modifier(xml_obj, modifier=3)
    score, input_x, edges, note_locations = get_input_from_xml(xml_modified, 'Chopin', None, stats['input_keys'], model.stats['graph_keys'], model.stats['stats'], device)
    save_folder = 'test_210716/'
    Path(save_folder).mkdir(exist_ok=True)
    for n, info in enumerate(zip(labels, selected_z)):
        for l in range(1):
            piece = info[1]
            mu = info[1]
            z = reparameterize(mu, torch.ones_like(mu)*0.25)
            # np.save(Path(save_folder + f'{score_file.name}_{n}_{l}.npy'), z.cpu().numpy())
            with torch.no_grad():
                outputs, perform_mu, perform_var, total_out_list = model(input_x, None, edges, note_locations, initial_z=z)

            save_path = Path(save_folder + f'randomize_{k}_{n}_{l}.mid')
            save_model_output_as_midi(outputs, save_path, score, model.stats['output_keys'], model.stats['stats'], note_locations, bool_pedal=args.boolPedal, disklavier=args.disklavier)


645
695
659
703
687
652
659
697
677
686


In [75]:
p_add = 0.15
p_shift = 0.1
p_del = 0.1
modifier = 1

weights = [p_add, p_shift, p_del] * modifier
weights.append(1 - sum(weights))
action = random.choices(['add', 'shift', 'del', 'pass'], weights=weights)
print(action)

['pass']
